In [1]:
import datetime
import time
import psycopg2

url = "hdfs://127.0.0.1:9000/socmed_input/"
out_url = "hdfs://127.0.0.1:9000/socmed_output/"

conn = psycopg2.connect(database="bigdatatest",
    host="localhost",
    user="geraldakbar",
    password='bigdata'
)

In [2]:
def normalizeTime(time):
    res = ''
    for i in range(len(time)):
        if i == 6 or i == 7:
            res += '0'
        elif i == 4:
            if int(time[i]) < 5:
                res += '0'
            else:
                res += '5'
        else:
            res += time[i] 
    return res

def parseTime(type, time):
    if type == "twitter":
        ts = t.strftime('%Y-%m-%d %H:%M:%S', t.strptime(time,'%a %b %d %H:%M:%S +0000 %Y'))
        return ts[:10]
    elif type == "instagram" or type == "other":
        time = str(datetime.datetime.fromtimestamp(int(time)).date())
        return time[:10]
    else:
        return time[:10] + ' ' + normalizeTime(time[11:19])

In [10]:
fb = spark.read.json(f"{url}facebook*.json").rdd

def map(line):
    result = []
    postDate = line["created_time"][0:10] + ' ' + normalizeTime(line["created_time"][11:19])
    comments = line["comments"]["data"]
    result.append((("facebook", postDate), 1))
    for comment in comments:
        date = comment["created_time"][0:10] + ' ' + normalizeTime(comment["created_time"][11:19])
        result.append((("facebook", date), 1))
    return result

def userMap(line):
    user = line["from"]["name"]
    postDate = line["created_time"][0:10] + ' ' + normalizeTime(line["created_time"][11:19])
    return (("facebook", postDate, user), 1)
    

cur = conn.cursor()
fbPost = fb.flatMap(map)
fbPost = fbPost.reduceByKey(lambda x, y: x + y)

user = fb.map(userMap).distinct().map(lambda x: ((x[0][0], x[0][1]), x[1]))
user = user.reduceByKey(lambda x, y: x + y)

fbPost = fbPost.join(user).map(lambda x: (x[0], x[1][0], x[1][1]))
fbPost = fbPost.map(lambda x: (x[0][0], x[0][1], x[1], x[2]))
test = fbPost.take(5)

insertToSQL(test,conn,cur)

In [14]:
!git pull

Updating 9882c11..e86c756
Fast-forward
 .../Facebook-checkpoint.ipynb                      |    4 +-
 .ipynb_checkpoints/Instagram-checkpoint.ipynb      |  156 +++
 .ipynb_checkpoints/Twitter-checkpoint.ipynb        |  130 ++
 .ipynb_checkpoints/Youtube-checkpoint.ipynb        |  148 ++
 .../spark_socmed_13520068-checkpoint.ipynb         | 1458 ++++++++++++++++++++
 Facebook.ipynb                                     |  162 +++
 Instagram.ipynb                                    |  156 +++
 Twitter.ipynb                                      |  130 ++
 Youtube.ipynb                                      |  148 ++
 consumer.py                                        |   28 +-
 producer.py                                        |    6 +-
 spark_socmed_13520068.ipynb                        |   16 +
 12 files changed, 2537 insertions(+), 5 deletions(-)
 rename Tugas 3 .ipynb => .ipynb_checkpoints/Facebook-checkpoint.ipynb (98%)
 create mode 100644 .ipynb_checkpoints/Instagram-checkpoint.ipynb


In [8]:
def insertToSQL(data,connection,cursor):
    for element in data:
        social_media = element[0]
        timestamp = element[1]
        count = element[2]
        uniquecount = element[3]

        query = f"""INSERT INTO social_media_data (social_media, timestamp, count, uniquecount)
                    VALUES ('{social_media}', '{timestamp}', {count}, {uniquecount})
                    ON CONFLICT (social_media, timestamp) DO UPDATE
                    SET count = CASE WHEN social_media_data.count IS NULL THEN 1 ELSE social_media_data.count + {count} END,
                    uniquecount = CASE WHEN social_media_data.uniquecount IS NULL THEN 1 ELSE social_media_data.uniquecount + {uniquecount} END,
                    updated_at = NOW();"""



        cursor.execute(query)

    connection.commit()
    